<p> <strong>Nama:</strong> Naufal Prima Yoriko</p>
<p> <strong>Email:</strong> primayoriko@gmail.com </p>
<p> <strong>Username:</strong> primayoriko </p>
<p> <strong>Dicoding Profile:</strong> https://www.dicoding.com/users/primayoriko </p>


# **Submission PaperRockScissors**

Sebelum menuju ke langkah pengerjaan, ini terdapat beberapa referensi yang saya pakai

1.   Neural Network Activation Function in Keras
     *   https://missinglink.ai/guides/neural-network-concepts/7-types-neural-network-activation-functions-right/
     *   https://keras.io/api/layers/activations/

2.   Loss Function in Keras
     *   https://neptune.ai/blog/keras-loss-functions
     *   https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
     *   https://keras.io/api/losses/
     *   https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

3.   Callback in Keras
     * https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/

4.   Optimization in Keras
     * https://keras.io/api/optimizers/ 

5.   Performance comparison
     * https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/
     * https://www.dlology.com/blog/quick-notes-on-how-to-choose-optimizer-in-keras/
     * https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/70253



Berikut ini adalah tahap-tahap pengerjaan yang saya lakukan dalam memeroleh hasil dari Image Processing menggunakan CNN (*Convolutional Neural Network*)

**Pertama**, mendownload file dari web dicoding dengan menggunakan utilitas `wget`

In [4]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-10-31 19:05:57--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  6.24MB/s    in 48s     

2020-10-31 19:06:46 (6.37 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



**Kedua**, melakukan ekstraksi dari file yang didownload dengan menggunakan `zipfile`

In [5]:
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Ketiga**, dengan menggunakan library tambahan yaitu `split-folder`, saya melakukan splitting dari file-file gambar menjadi training (`train`) dan validation (`val`) data. 

**Note:** referensi split saya dapat dari forum dicoding, saya rasa karena dari forum, bukan website luar, tidak apa-apa untuk diikuti

In [6]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/tmp/rockpaperscissors/rps-cv-images', output='/tmp/rockpaperscissors/splitted', seed=1337, ratio=(.6, .4))

Copying files: 2188 files [00:00, 2867.24 files/s]


**Keempat**, saya mencatat direktori dari tiap tipe data, `train` dan `val` data

In [7]:
base_dir = '/tmp/rockpaperscissors/splitted'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
print(train_dir)
print(validation_dir)

/tmp/rockpaperscissors/splitted/train
/tmp/rockpaperscissors/splitted/val


**Kelima**, mencatat direktori dari tiap-tiap kategori gambar dan juga mencatat total file image di tiap segmennya, untuk berjaga semisal akan dibutuhkan kedepannya.

In [8]:
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')
train_rock_dir = os.path.join(train_dir, 'rock')

validation_paper_dir = os.path.join(validation_dir, 'paper')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')
validation_rock_dir = os.path.join(validation_dir, 'rock')

train_paper = len(os.listdir(train_paper_dir))
train_scissors = len(os.listdir(train_scissors_dir))
train_rock = len(os.listdir(train_rock_dir))

validation_paper = len(os.listdir(validation_paper_dir))
validation_scissors = len(os.listdir(validation_scissors_dir))
validation_rock = len(os.listdir(validation_rock_dir))

total_train = train_paper + train_scissors + train_rock
total_validation = validation_paper + validation_scissors + validation_rock

print(total_train)
print(total_validation)

1312
876


**Keenam**, pembuatan generator image dari gambar-gambar yang tersedia, dengan menggunakan `ImageDataGenerator` dari Keras. Fungsi dari image data generator ini adalah untuk membantu mengklasifikasikan suatu gambar ke kategoti tertentu karena pada gambar mungkin ada faktor tertentu yang membedakan tampilan (diluar karena perbedaan objek), seperti jarak/ukuran, orientasi, gradasi warna, dll.

Disini kode yang digunakan kurang lebih seperti yang ada di modul `Latihan Membuat Model Klasifikasi Gambar` di kelas ini.

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    shear_range=0.2,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode = 'nearest'
                  )

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    shear_range=0.2,
                    # width_shift_range=0.2,
                    # height_shift_range=0.2,
                    # zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode = 'nearest'
                  )

**Keenam**, Membuat pengaturan dimensi image, class, dan juga flow dari transfer data image dalam pembuatan model di tiap batchnya.

Disini kode yang digunakan kurang lebih seperti yang ada di modul `Latihan Membuat Model Klasifikasi Gambar` di kelas ini. Namun, terdapat penyesuaian size tiap batch-nya sekaligus `class_mode` nya.

In [20]:
from math import ceil

# Diasumsikan akan dibuat dibuat 25 step tiap epoch
steps_per_epoch = 25

# Maka berikut ini jumlah file di tiap flow
train_per_batch = ceil(total_train/steps_per_epoch)
val_per_batch = ceil(total_validation/steps_per_epoch)
print(train_per_batch)
print(val_per_batch)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=train_per_batch,
                    class_mode='categorical'
                  )
 
validation_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(150, 150),
                          batch_size=val_per_batch,
                          class_mode='categorical'
                        )

53
36
Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


**Ketujuh**, membuat struktur dari CNN. CNN yang dibuat disini, sesuai spesifikasi menggunakan model sequensial, yang susunannya dibebaskan (asal memiliki > 1 hidden layer).

Disini kode/model yang digunakan kurang lebih seperti yang ada di modul `Latihan Membuat Model Klasifikasi Gambar` di kelas ini. Namun, ........

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dense(512, activation= 'relu'))
model.add(Flatten())
model.add(Dense(3, activation= 'softmax'))

**Kedelapan**,

In [12]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])

In [ ]:
import time

start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=6,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

In [54]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

# opt = Adam(amsgrad=True)
opt = SGD(
    lr=0.01, momentum=0.9, nesterov=False
)

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

Epoch 1/12
25/25 - 60s - loss: 1.0957 - accuracy: 0.3445 - val_loss: 1.0756 - val_accuracy: 0.4833
Epoch 2/12
25/25 - 60s - loss: 1.0693 - accuracy: 0.4268 - val_loss: 0.9876 - val_accuracy: 0.5167
Epoch 3/12
25/25 - 63s - loss: 0.9753 - accuracy: 0.5488 - val_loss: 0.7969 - val_accuracy: 0.7944
Epoch 4/12
25/25 - 60s - loss: 0.8593 - accuracy: 0.5960 - val_loss: 0.5855 - val_accuracy: 0.7556
Epoch 5/12
25/25 - 59s - loss: 0.6957 - accuracy: 0.7096 - val_loss: 0.4364 - val_accuracy: 0.9111
Epoch 6/12
25/25 - 59s - loss: 0.5474 - accuracy: 0.7843 - val_loss: 0.4002 - val_accuracy: 0.8778
Epoch 7/12
25/25 - 59s - loss: 0.4315 - accuracy: 0.8377 - val_loss: 0.2308 - val_accuracy: 0.9444
Epoch 8/12
25/25 - 59s - loss: 0.4448 - accuracy: 0.8247 - val_loss: 0.2580 - val_accuracy: 0.9500
Epoch 9/12
25/25 - 60s - loss: 0.3636 - accuracy: 0.8659 - val_loss: 0.2153 - val_accuracy: 0.9556
Epoch 10/12
25/25 - 60s - loss: 0.2979 - accuracy: 0.9055 - val_loss: 0.1096 - val_accuracy: 0.9667
Epoch 11/

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

# opt = Adam(amsgrad=True)
opt = SGD(
    lr=0.01, momentum=0.9, nesterov=True
)

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))



Epoch 1/12
25/25 - 63s - loss: 1.0925 - accuracy: 0.4108 - val_loss: 1.0723 - val_accuracy: 0.4222
Epoch 2/12
25/25 - 60s - loss: 1.0577 - accuracy: 0.4436 - val_loss: 0.9289 - val_accuracy: 0.6111
Epoch 3/12
25/25 - 60s - loss: 0.9754 - accuracy: 0.5389 - val_loss: 0.7267 - val_accuracy: 0.8222
Epoch 4/12
25/25 - 60s - loss: 0.8009 - accuracy: 0.6730 - val_loss: 1.2443 - val_accuracy: 0.4056
Epoch 5/12
25/25 - 59s - loss: 1.1525 - accuracy: 0.4200 - val_loss: 0.9915 - val_accuracy: 0.6056
Epoch 6/12
25/25 - 59s - loss: 0.8816 - accuracy: 0.6166 - val_loss: 0.6166 - val_accuracy: 0.7389
Epoch 7/12
25/25 - 60s - loss: 0.6576 - accuracy: 0.7325 - val_loss: 0.3380 - val_accuracy: 0.9056
Epoch 8/12


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=True
# )

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

In [51]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=True
# )

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=7,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

Epoch 1/7
25/25 - 60s - loss: 0.1945 - accuracy: 0.3788 - val_loss: 0.1761 - val_accuracy: 0.6778
Epoch 2/7
25/25 - 60s - loss: 0.1786 - accuracy: 0.4825 - val_loss: 0.1658 - val_accuracy: 0.5944
Epoch 3/7
25/25 - 59s - loss: 0.1618 - accuracy: 0.5846 - val_loss: 0.0926 - val_accuracy: 0.9611
Epoch 4/7
25/25 - 60s - loss: 0.1036 - accuracy: 0.7973 - val_loss: 0.0528 - val_accuracy: 0.9444
Epoch 5/7
25/25 - 63s - loss: 0.0695 - accuracy: 0.8880 - val_loss: 0.0425 - val_accuracy: 0.9500
Epoch 6/7
25/25 - 60s - loss: 0.0561 - accuracy: 0.9177 - val_loss: 0.0456 - val_accuracy: 0.9667
Epoch 7/7
25/25 - 59s - loss: 0.0376 - accuracy: 0.9428 - val_loss: 0.0172 - val_accuracy: 0.9833
--- Finished in 441.4690737724304 seconds ---


In [47]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential(name="simple_sequential")
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True, lr=0.001)
# opt = SGD(
#     lr=0.01, momentum=0.9, nesterov=False
# )

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=7,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))

Epoch 1/7
25/25 - 61s - loss: 1.1147 - accuracy: 0.3445 - val_loss: 1.0836 - val_accuracy: 0.3556
Epoch 2/7
25/25 - 60s - loss: 0.9432 - accuracy: 0.5366 - val_loss: 0.5360 - val_accuracy: 0.8389
Epoch 3/7
25/25 - 60s - loss: 0.5590 - accuracy: 0.7759 - val_loss: 0.3567 - val_accuracy: 0.8722
Epoch 4/7
25/25 - 59s - loss: 0.4956 - accuracy: 0.7904 - val_loss: 0.2802 - val_accuracy: 0.9278
Epoch 5/7
25/25 - 63s - loss: 0.3847 - accuracy: 0.8651 - val_loss: 0.1175 - val_accuracy: 0.9778
Epoch 6/7
25/25 - 60s - loss: 0.3941 - accuracy: 0.8552 - val_loss: 0.1961 - val_accuracy: 0.9500
Epoch 7/7
25/25 - 59s - loss: 0.2763 - accuracy: 0.9009 - val_loss: 0.1356 - val_accuracy: 0.9722
--- Finished in 443.3176734447479 seconds ---


Dari sini, didapati akurasi sebesar **99.44%** pada `val` data, sedangkan akurasi sebesar **98.02%** pada `train` data sendiri. Untuk waktu eksekusi, training dan validasi data memakan waktu selama **1411 s** ~ **24 min**

**Kesembilan**, ini opsional, semisal anda ingin mencoba memasukkan untuk menguji suatu image tangan guna diklasifikasikan ke dalam salah satu kategori (paper, rock, scissors).

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# %matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  # imgplot = plt.imshow(img)
  # x = image.img_to_array(img)
  # x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  print(fn)
  print(classes)

  # if classes==0:
  #   print('clean')
  # else:
  #   print('messy')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16


prior = VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(150, 150, 3)
)

model = Sequential(name="simple_sequential")
model.add(prior)
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation= 'sigmoid'))
# model.add(Dense(3, activation= 'softmax'))

import tensorflow_addons as tfa
from keras.optimizers import SGD, Adam

opt = Adam(amsgrad=True)

model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = opt,
              metrics=['accuracy'])
start_time = time.time()
schema = model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=7,
        validation_data=validation_generator,
        validation_steps=5,
        verbose=2
      )
print("--- Finished in %s seconds ---" % (time.time() - start_time))